In [48]:
import pandas as pd

In [49]:
df = pd.read_csv('raw_data2.csv', encoding='utf-8-sig', index_col=0)
df.head()

,region,education,jobtype,cptype,sales,employees,aversalary,capital,pros
0,서울,4,3,2,180908204,"2,036",9516,7781959,안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-복지의 끝판왕-매출 1조 클럽...
1,서울,4,3,3,60628188,"5,415",2858,1297880,성장판 열린 회사-복지의 끝판왕-HOT한 인기기업-보너스로 행복해요♥-장기근속수당제...
2,서울,4,1,3,60628188,"5,415",-,1297880,안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-복지의 ...
3,서울,0,1,3,60628188,"5,415",-,1297880,성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-매출 1조 클럽-HOT한 인...
4,서울,0,3,3,32879127,"3,241",5739,208129,성장판 열린 회사-채용에 적극적인 회사-HOT한 인기기업-지방에 위치한 대기업


In [50]:
tags_encoded = pd.read_csv('tags_encoded.csv', encoding='utf-8-sig', index_col=0)
# tags_encoded.head()
tags_score = dict(zip(tags_encoded['tags'], tags_encoded['score']))
tags_score

{'안정적인 장수 회사': 5,
 '성장판 열린 회사': 3,
 '구직자 인기기업': 5,
 '채용에 적극적인 회사': 3,
 '복지의 끝판왕': 1,
 '매출 1조 클럽': 5,
 '매출 상위 10%': 5,
 'HOT한 인기기업': 5,
 '아침을 왜 굶어요?': 1,
 '평균연봉 6천이상': 3,
 '신입연봉이 4천이상': 3,
 '스타트업 초봉 3,500만원↗': 3,
 '지방에 위치한 대기업': 5,
 '자기계발비 지원 기업': 1,
 '보너스로 행복해요♥': 1,
 '내 출퇴근은 내 맘대로': 1,
 '아낌없이 주는 커피': 1,
 '아이와 함께 출근해요': 1,
 '잘나가는 의료·바이오기업': 5,
 '주 4일 근무': 3,
 '집 걱정 안 하기': 3,
 '거침없이 성장하는 회사': 3,
 '매출 1000대 기업': 5,
 '인크루트 인증 우수기업': 3,
 '대기업 in 서울': 5,
 '공기업 in 서울': 5,
 '장기근속수당제공 기업': 3,
 '육아휴직 제공 기업': 1,
 'KOSPI 상장': 5,
 'KOSDAQ 상장': 5,
 '빵빵한 구내식당': 1,
 '휴식까지 챙겨줘요!': 1,
 'KONEX 상장': 3}

In [51]:
def encodeTagsByScore(tags: str):
    if isinstance(tags, float): return tags
    total_score = 0
    for tag in tags.split('-'):
        if tag !=  '':
            total_score += tags_score[tag]
    return total_score

df['pros_encoded'] = df['pros'].apply(encodeTagsByScore)
df['pros_encoded']
# df.head()

0      50.0
1      18.0
2      49.0
3      29.0
4      16.0
       ... 
165    28.0
166    36.0
167    36.0
168    36.0
169    36.0
Name: pros_encoded, Length: 170, dtype: float64

In [52]:
df

,region,education,jobtype,cptype,sales,employees,aversalary,capital,pros,pros_encoded
0,서울,4,3,2,180908204,"2,036",9516,7781959,안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-복지의 끝판왕-매출 1조 클럽...,50.0
1,서울,4,3,3,60628188,"5,415",2858,1297880,성장판 열린 회사-복지의 끝판왕-HOT한 인기기업-보너스로 행복해요♥-장기근속수당제...,18.0
2,서울,4,1,3,60628188,"5,415",-,1297880,안정적인 장수 회사-성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-복지의 ...,49.0
3,서울,0,1,3,60628188,"5,415",-,1297880,성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-매출 1조 클럽-HOT한 인...,29.0
4,서울,0,3,3,32879127,"3,241",5739,208129,성장판 열린 회사-채용에 적극적인 회사-HOT한 인기기업-지방에 위치한 대기업,16.0
...,...,...,...,...,...,...,...,...,...,...
165,제주,4,3,3,92194123,"3,033",3277,2400000,성장판 열린 회사-구직자 인기기업-채용에 적극적인 회사-거침없이 성장하는 회사-매출...,28.0
166,제주,0,3,3,92194123,"3,033",-,2400000,구직자 인기기업-채용에 적극적인 회사-복지의 끝판왕-HOT한 인기기업-평균연봉 6천...,36.0
167,제주,0,3,3,92194123,"3,033",-,2400000,구직자 인기기업-채용에 적극적인 회사-복지의 끝판왕-HOT한 인기기업-평균연봉 6천...,36.0
168,제주,0,3,3,92194123,"3,033",-,2400000,구직자 인기기업-채용에 적극적인 회사-복지의 끝판왕-HOT한 인기기업-평균연봉 6천...,36.0
